In [1]:
import trafilatura
import requests
import lxml
from elasticsearch import Elasticsearch, helpers


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
cafef = {
    "life": ["https://cafef.vn/song.chn",
             "https://cafef.vn/lifestyle.chn"],
    "news": ["https://cafef.vn/thoi-su.chn"],
    "world": ["https://cafef.vn/tai-chinh-quoc-te.chn"]
}


In [3]:
cafefLinks = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in cafef.items():
    for url in value:
        response = requests.get(url)
        tree = lxml.html.fromstring(response.content)
        title_elem = tree.cssselect('h3 a')
        for elem in title_elem:
            cafefLinks[key].append("https://cafef.vn"+elem.get('href'))

print(cafefLinks)

{'life': ['https://cafef.vn/5-thoi-quen-khien-dan-ong-lao-hoa-nhanh-can-khong-noi-tu-30-tuoi-rat-nen-chu-y-bo-sung-3-thuc-pham-quen-thuoc-de-niu-giu-tuoi-tre-20220406150231827.chn', 'https://cafef.vn/bi-quyet-song-tho-nam-o-trai-tim-thu-hai-cua-co-the-la-ang-ten-bao-benh-nuoi-song-mot-nua-so-mach-mau-nhung-nhieu-nguoi-khong-quan-tam-20220405215952369.chn', 'https://cafef.vn/trong-lang-tan-thuy-hoang-co-mot-ngon-den-chay-ngan-nam-khong-tat-nguyen-lieu-lam-nen-anh-sang-bat-tu-khien-nguoi-doi-phai-kinh-hai-20220406150656494.chn', 'https://cafef.vn/chang-trai-dung-xe-tai-cho-260000-dong-xu-di-mua-mo-to-mo-uoc-bay-lau-tien-vat-va-tich-cop-trong-hon-3-nam-suyt-bi-nguoi-ban-hang-tu-choi-20220406151835965.chn', 'https://cafef.vn/tron-mat-ong-cung-loai-hat-nay-se-thanh-sieu-thuc-pham-tri-benh-da-day-bao-ve-xuong-va-rat-tot-cho-benh-nhan-tieu-duong-20220406134901942.chn', 'https://cafef.vn/ly-giai-loi-don-an-nhung-thuc-pham-nay-cung-nhau-se-gay-chet-nguoi-20220406155156106.chn', 'https://cafef.v

In [4]:
cafefContent = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in cafefLinks.items():
    for url in value:
        if(url.startswith('https://cafef.vn/')):
            id = url.split('-')[-1].replace('.chn', '')
            downloaded = trafilatura.fetch_url(url)
            tree = lxml.html.fromstring(str(downloaded))
            date_elem = tree.cssselect(".pdate")[0]
            content = trafilatura.extract(downloaded)
            content = content.replace('\n', ' ')
            item = {"domain": "https://cafef.vn", "URL": url,"content": content, "date": date_elem.text, "id": id}
            cafefContent[key].append(item)

print(len(cafefContent["life"]) + len(cafefContent["news"]) + len(cafefContent["world"]))

56


In [5]:
client = Elasticsearch("http://localhost:9200")

client.info()


/home/hplong/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'HoangPhiLong-20182651',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'jF2zfdDdQqueQik2u6h9Ug',
 'version': {'number': '7.17.2',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': 'de7261de50d90919ae53b0eff9413fd7e5307301',
  'build_date': '2022-03-28T15:12:21.446567561Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [6]:
def gendata():
    for key,value in cafefContent.items():
         for item in value:
            temp_item = {k: item[k] for k in set(list(item.keys())) - set(['id'])}
            temp_item['category'] = key
            client.index(index="journal", id=item['id'], document=temp_item)

gendata()